# Vertex AI SDK for Python: AutoML training video object tracking model for batch prediction


This demo demonstrates how to use the Vertex AI SDK for Python to create video object tracking models and do batch prediction using a Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) model.

Learn more about [Object tracking for video data](https://cloud.google.com/vertex-ai/docs/training-overview#object_tracking_for_videos).

### Objective

This demo shows how to create an AutoML video object tracking model from a Python script, and then do a batch prediction using the Vertex AI SDK. Alternatively, you can create and deploy models using the `gcloud` command-line tool or online using the Cloud Console.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI

The steps performed include:

- Create a Vertex `Dataset` resource.
- Train the model.
- View the model evaluation.
- Make a batch prediction.

There is one key difference between using batch prediction and using online prediction:

* Prediction Service: Does an on-demand prediction for the entire set of instances (i.e., one or more data items) and returns the results in real-time.

* Batch Prediction Service: Does a queued (batch) prediction for the entire set of instances in the background and stores the results in a Cloud Storage bucket when ready.

### Dataset

The dataset used for this tutorial is the [Traffic](https://storage.googleapis.com/automl-video-demo-data/traffic_videos/traffic_videos_labels.csv) dataset.

In [ ]:
PROJECT_ID = "ibnd-argls-cstmr-demos"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
# BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"} ibnd-argls-cstmr-demos-vertexai-vision/06-sdk-objct-tracking
BUCKET_URI = "gs://ibnd-argls-cstmr-demos-vertexai-vision/06-sdk-objct-tracking"

### Import libraries and define constants

In [ ]:
import json
import os

import google.cloud.aiplatform as aiplatform
from google.cloud import storage

## Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### Start creating the AutoML video object tracking model.

#### Location of Cloud Storage training data.

Now set the variable `IMPORT_FILE` to the location of the CSV index file in Cloud Storage.

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/ai-platform-unified/video/traffic/traffic_videos_labels.csv"

#### Quick peek at your data

Start by doing a quick peek at the data. You count the number of examples by counting the number of rows in the CSV index file  (`wc -l`) and then peek at the first few rows.

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

! gsutil cat $FILE | head

Number of Examples 685
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,sedan,1565750291672021,11.933333,0.509205,0.594283,,,0.728737,0.760959,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672171,17.566666,0.761241,0.498466,,,0.948839,0.668524,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672223,20.433333,0.000000,0.465235,,,0.142638,0.665644,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672347,25.766666,0.486523,0.592331,,,0.720611,0.776687,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672575,28.966666,0.578534,0.652778,,,0.828647,0.862967,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672549,28.966666,0.000000,0.518571,,,0.148841,0.737677,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672599,28.966666,0.106979,0.458078,,,0.377877,0.678937,,

### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the `VideoDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.

This operation may take several minutes.

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name="Traffic",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.object_tracking,
)

print(dataset.resource_name)

Creating VideoDataset
Create VideoDataset backing LRO: projects/998979163436/locations/us-central1/datasets/3006211025386078208/operations/8341554846565924864
VideoDataset created. Resource name: projects/998979163436/locations/us-central1/datasets/3006211025386078208
To use this VideoDataset in another session:
ds = aiplatform.VideoDataset('projects/998979163436/locations/us-central1/datasets/3006211025386078208')
Importing VideoDataset data: projects/998979163436/locations/us-central1/datasets/3006211025386078208
Import VideoDataset data backing LRO: projects/998979163436/locations/us-central1/datasets/3006211025386078208/operations/5241389453074759680
VideoDataset data imported. Resource name: projects/998979163436/locations/us-central1/datasets/3006211025386078208
projects/998979163436/locations/us-central1/datasets/3006211025386078208


### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLVideoTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `prediction_type`: The type task to train the model for.
  - `classification`: A video classification model.
  - `object_tracking`: A video object tracking model.
  - `action_recognition`: A video action recognition model.


In [ ]:
job = aiplatform.AutoMLVideoTrainingJob(
    display_name="traffic",
    prediction_type="object_tracking",
)

print(job)

#### Run the training pipeline

Next, you start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take upto 4 hours.

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name="traffic",
    training_fraction_split=0.8,
    test_fraction_split=0.2,
)

View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/1255675745305362432?project=998979163436
AutoMLVideoTrainingJob projects/998979163436/locations/us-central1/trainingPipelines/1255675745305362432 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLVideoTrainingJob projects/998979163436/locations/us-central1/trainingPipelines/1255675745305362432 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLVideoTrainingJob projects/998979163436/locations/us-central1/trainingPipelines/1255675745305362432 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLVideoTrainingJob projects/998979163436/locations/us-central1/trainingPipelines/1255675745305362432 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLVideoTrainingJob projects/998979163436/locations/us-central1/trainingPipelines/1255675745305362432 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLVideoTrainingJob projects/998979163436/locations/us-central1/trainingPipeline

## Review model evaluation scores
After your model has finished training, you can review the evaluation scores for it.

First, you need to get a reference to the new model. As with datasets, you can either use the reference to the model variable you created when you deployed the model or you can list all of the models in your project.

In [ ]:
model_evaluations = model.list_model_evaluations()
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

resource name: projects/998979163436/locations/us-central1/models/5767666364382707712@1/evaluations/875566002828476416


In [ ]:
# Print the evaluation metrics
for evaluation in model_evaluations:
    evaluation = evaluation.to_dict()
    print("Model's evaluation metrics from Training:\n")
    metrics = evaluation["metrics"]
    for metric in metrics.keys():
        print(f"metric: {metric}, value: {metrics[metric]}\n")

Model's evaluation metrics from Training:

metric: boundingBoxMeanAveragePrecision, value: 0.30644187

metric: boundingBoxMetrics, value: [{'iouThreshold': 0.5, 'confidenceMetrics': [{'precision': 0.3993994, 'recall': 1.0, 'f1Score': 0.57081544, 'confidenceThreshold': 0.0}, {'recall': 0.14285715, 'confidenceThreshold': 0.058405496, 'f1Score': 0.10795455, 'precision': 0.08675799}, {'confidenceThreshold': 0.059411008, 'recall': 0.14285715, 'precision': 0.08715596, 'f1Score': 0.10826211}, {'recall': 0.14285715, 'precision': 0.08755761, 'f1Score': 0.108571425, 'confidenceThreshold': 0.061475746}, {'recall': 0.14285715, 'precision': 0.08796296, 'f1Score': 0.108882524, 'confidenceThreshold': 0.062099054}, {'confidenceThreshold': 0.063024625, 'recall': 0.14285715, 'f1Score': 0.109195404, 'precision': 0.0883721}, {'precision': 0.088785045, 'recall': 0.14285715, 'f1Score': 0.10951009, 'confidenceThreshold': 0.0640149}, {'f1Score': 0.10982659, 'recall': 0.14285715, 'precision': 0.089201875, 'con

## Send a batch prediction request

Send a batch prediction to your deployed model.

### Get test item(s)

Now do a batch prediction to your Vertex model. You will use arbitrary examples out of the dataset as a test items. Don't be concerned that the examples were likely used in training the model -- we just want to demonstrate how to make a prediction.

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2
cols_1 = test_items[0].split(",")
cols_2 = test_items[1].split(",")
if len(cols_1) > 12:
    test_item_1 = str(cols_1[1])
    test_item_2 = str(cols_2[1])
    test_label_1 = str(cols_1[2])
    test_label_2 = str(cols_2[2])
else:
    test_item_1 = str(cols_1[0])
    test_item_2 = str(cols_2[0])
    test_label_1 = str(cols_1[1])
    test_label_2 = str(cols_2[1])

### Make a batch input file

Now make a batch input file, which you store in your local Cloud Storage bucket. The batch input file can be either CSV or JSONL. You will use JSONL in this tutorial. For JSONL file, you make one dictionary entry per line for each video. The dictionary contains the key/value pairs:

- `content`: The Cloud Storage path to the video.
- `mimeType`: The content type. In our example, it is a `avi` file.
- `timeSegmentStart`: The start timestamp in the video to do prediction on. *Note*, the timestamp must be specified as a string and followed by s (second), m (minute) or h (hour).
- `timeSegmentEnd`: The end timestamp in the video to do prediction on.

In [ ]:
test_filename = "test.jsonl"
gcs_input_uri = BUCKET_URI + "/test.jsonl"
# making data_1 and data_2 variables using the structure mentioned above
data_1 = {
    "content": test_item_1,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}

data_2 = {
    "content": test_item_2,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}

# getting reference to bucket
bucket = storage.Client(project=PROJECT_ID).bucket(BUCKET_URI.replace("gs://", ""))

# creating a blob
blob = bucket.blob(blob_name=test_filename)

# creating data variable
data = json.dumps(data_1) + "\n" + json.dumps(data_2) + "\n"

# uploading data variable content to bucket
blob.upload_from_string(data)

# printing path of uploaded file
print(gcs_input_uri)

# printing content of uploaded file
! gsutil cat $gcs_input_uri

NotFound: 404 POST https://storage.googleapis.com/upload/storage/v1/b/ibnd-argls-cstmr-demos-vertexai-vision/06-sdk-objct-tracking/o?uploadType=multipart: Not Found: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>)

### Make the batch prediction request

Now that your Model resource is trained, you can make a batch prediction by invoking the batch_predict() method, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `gcs_source`: A list of one or more batch request input files.
- `gcs_destination_prefix`: The Cloud Storage location for storing the batch prediction resuls.
- `sync`: If set to True, the call will block while waiting for the asynchronous batch job to complete.

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="traffic",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

### Wait for completion of batch prediction job

Next, wait for the batch job to complete. Alternatively, one can set the parameter `sync` to `True` in the `batch_predict()` method to block until the batch prediction job is completed.

In [ ]:
batch_predict_job.wait()

### Get the predictions

Next, get the results from the completed batch prediction job.

The results are written to the Cloud Storage output bucket you specified in the batch prediction request. You call the method iter_outputs() to get a list of each Cloud Storage file generated with the results. Each file contains one or more prediction requests in a JSON format:

- `content`: The prediction request.
- `prediction`: The prediction response.
 - `id`: The internal assigned unique identifiers for each prediction request.
 - `displayName`: The class names for the predicted label.
 - `confidences`: The predicted confidence, between 0 and 1, per class label.
 - `timeSegmentStart`: The time offset in the video to the start of the video sequence.
 - `timeSegmentEnd`: The time offset in the video to the end of the video sequence.
 - `frames`: Location with frames of the tracked object.

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}".replace(
        BUCKET_URI + "/", ""
    )
    data = bucket.get_blob(gfile_name).download_as_string()
    data = json.loads(data)
    print(data)

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Model
- AutoML Training Job
- Batch Job
- Cloud Storage Bucket

In [ ]:
# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI